We are going to extract data from redshift and carry out the following transformation tasks
- remove all cases where customer id is missing (yes)
- remove all duplicate data (yes)
- replace missing stock description and where ? with Unknown (yes)
- fix data type (yes)
- remove certain stock codes (yes)
- add description to the table (yes)

In [54]:
# import the libraries we need

import psycopg2
import pandas as pd

import boto3
from io import StringIO

import warnings
warnings.filterwarnings("ignore")


## Extracting Data

In [3]:
# NEVER share passwords

dbname = "dev"
host = ""
port = 5439
user = ""
password = ""

In [4]:
# connect to redshift

def connect_to_redshift(dbname, host, port, user, password):
    """connect to redshift"""

    connect = psycopg2.connect(
            dbname=dbname, host=host, port=port, user=user, password=password
        )

    cursor = connect.cursor()
    
    print("connection to redshift made")
    
    return connect, cursor

In [5]:
connect, cursor = connect_to_redshift(dbname, host, port, user, password)

connection to redshift made


In [7]:
query = """select *
from bootcamp1.stock_description
limit 10
"""

pd.read_sql(query, connect)

,stock_code,description
0,10002,INFLATABLE POLITICAL GLOBE
1,10080,GROOVY CACTUS INFLATABLE
2,10120,DOGGY RUBBER
3,10123C,HEARTS WRAPPING TAPE
4,10124A,SPOTS ON RED BOOKCOVER TAPE
5,10124G,ARMY CAMO BOOKCOVER TAPE
6,10125,MINI FUNKY DESIGN TAPES
7,10133,COLOURING PENCILS BROWN TUBE
8,10135,COLOURING PENCILS BROWN TUBE
9,11001,ASSTD DESIGN RACING CAR PEN


In [22]:
# extracts data from redshift
# joins the description field to the online transactions table
# removes invoices where customer id is blank
# removes invoices where stock code is in bank charges, postages etc...

query = """
select o.*,
       s.description
from bootcamp1.online_transactions o
left join bootcamp1.stock_description s on o.stock_code = s.stock_code
where o.customer_id <> ''
    and o.stock_code not in ('BANK CHARGES', 'POSTAGE', 'D', 'M', 'CRUK')
"""

online_trans_w_desc = pd.read_sql(query, connect)
online_trans_w_desc.head()

,invoice,stock_code,quantity,invoice_date,price,customer_id,country,description
0,536365,84029G,6,2010-12-01 08:26:00,3.39,u1785,United Kingdom,KNITTED UNION FLAG HOT WATER BOTTLE
1,536366,22633,6,2010-12-01 08:28:00,1.85,u1785,United Kingdom,HAND WARMER UNION JACK
2,536368,22912,3,2010-12-01 08:34:00,4.95,u13047,United Kingdom,YELLOW COAT RACK PARIS FASHION
3,536367,22748,6,2010-12-01 08:34:00,2.10,u13047,United Kingdom,POPPY'S PLAYHOUSE KITCHEN
4,536367,22623,3,2010-12-01 08:34:00,4.95,u13047,United Kingdom,BOX OF VINTAGE JIGSAW BLOCKS


In [24]:
online_trans_w_desc[online_trans_w_desc.customer_id == '']

# we have no cases of missing customer ids

,invoice,stock_code,quantity,invoice_date,price,customer_id,country,description


In [20]:
type(online_trans_w_desc)

pandas.core.frame.DataFrame

In [21]:
online_trans_w_desc.shape

# dropped from 54k rows to 41k rows, and increased the number of columns by ome

(414119, 8)

## Transforming Data

### Removing duplicated data

In [26]:
# task 1 - Remove duplicated data

online_trans_w_desc.duplicated().sum()

5303

In [28]:
online_trans_w_desc[online_trans_w_desc.duplicated()]

,invoice,stock_code,quantity,invoice_date,price,customer_id,country,description
319,536412,22327,1,2010-12-01 11:49:00,2.95,u1792,United Kingdom,ROUND SNACK BOXES SET OF 4 SKULLS
327,536412,22273,1,2010-12-01 11:49:00,2.95,u1792,United Kingdom,FELTCRAFT DOLL MOLLY
332,536412,21708,1,2010-12-01 11:49:00,4.95,u1792,United Kingdom,FOLDING UMBRELLA CREAM POLKADOT
334,536412,85184C,1,2010-12-01 11:49:00,2.95,u1792,United Kingdom,SET 4 VALENTINE DECOUPAGE HEART BOX
395,536464,21992,1,2010-12-01 12:23:00,2.95,u17968,United Kingdom,VINTAGE PAISLEY STATIONERY SET
...,...,...,...,...,...,...,...,...
413936,581471,21411,2,2011-12-08 19:29:00,1.95,u14702,United Kingdom,GINGHAM HEART DOORSTOP RED
414044,581514,22075,24,2011-12-09 11:20:00,0.39,u17754,United Kingdom,6 RIBBONS ELEGANT CHRISTMAS
414055,581538,23275,1,2011-12-09 11:34:00,1.25,u14446,United Kingdom,SET OF 3 HANGING OWLS OLLIE BEAK
414060,581538,22068,1,2011-12-09 11:34:00,0.39,u14446,United Kingdom,BLACK PIRATE TREASURE CHEST


In [30]:
# code to only keep the first appearance of a duplicated row

online_trans_cleaned = online_trans_w_desc.drop_duplicates(keep = 'first')
online_trans_cleaned.head()

,invoice,stock_code,quantity,invoice_date,price,customer_id,country,description
0,536365,84029G,6,2010-12-01 08:26:00,3.39,u1785,United Kingdom,KNITTED UNION FLAG HOT WATER BOTTLE
1,536366,22633,6,2010-12-01 08:28:00,1.85,u1785,United Kingdom,HAND WARMER UNION JACK
2,536368,22912,3,2010-12-01 08:34:00,4.95,u13047,United Kingdom,YELLOW COAT RACK PARIS FASHION
3,536367,22748,6,2010-12-01 08:34:00,2.10,u13047,United Kingdom,POPPY'S PLAYHOUSE KITCHEN
4,536367,22623,3,2010-12-01 08:34:00,4.95,u13047,United Kingdom,BOX OF VINTAGE JIGSAW BLOCKS


In [31]:
# no duplicated rows of data anymore

online_trans_cleaned.duplicated().sum()

0

In [55]:
print(online_trans_w_desc.shape)
print(online_trans_cleaned.shape)

(414119, 8)
(408816, 8)


### Fixing the invoice date data type

In [32]:
online_trans_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 408816 entries, 0 to 414118
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   invoice       408816 non-null  object 
 1   stock_code    408816 non-null  object 
 2   quantity      408816 non-null  int64  
 3   invoice_date  408816 non-null  object 
 4   price         408816 non-null  float64
 5   customer_id   408816 non-null  object 
 6   country       408816 non-null  object 
 7   description   407644 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 28.1+ MB


In [56]:
# transformation task # 2 - fix the invoice data

online_trans_cleaned.invoice_date = pd.to_datetime(online_trans_cleaned.invoice_date, 
                                                   format = "%Y-%m-%d")
online_trans_cleaned.invoice_date

0        2010-12-01 08:26:00
1        2010-12-01 08:28:00
2        2010-12-01 08:34:00
3        2010-12-01 08:34:00
4        2010-12-01 08:34:00
                 ...        
414114   2011-12-09 12:49:00
414115   2011-12-09 12:50:00
414116   2011-12-09 12:50:00
414117   2011-12-09 12:50:00
414118   2011-12-09 12:50:00
Name: invoice_date, Length: 408816, dtype: datetime64[ns]

In [57]:
online_trans_cleaned.info()

# the invoice date is now a data type datetime64

<class 'pandas.core.frame.DataFrame'>
Int64Index: 408816 entries, 0 to 414118
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   invoice       408816 non-null  object        
 1   stock_code    408816 non-null  object        
 2   quantity      408816 non-null  int64         
 3   invoice_date  408816 non-null  datetime64[ns]
 4   price         408816 non-null  float64       
 5   customer_id   408816 non-null  object        
 6   country       408816 non-null  object        
 7   description   408816 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 28.1+ MB


### Replacing missing description values with Unknown 

In [58]:
online_trans_cleaned.isna().sum()

invoice         0
stock_code      0
quantity        0
invoice_date    0
price           0
customer_id     0
country         0
description     0
dtype: int64

In [59]:
# replace missing descriptions with Unknown

online_trans_cleaned.description.fillna("Unknown", inplace = True)

In [60]:
# check there are no missing descriptions

online_trans_cleaned.isnull().sum()

invoice         0
stock_code      0
quantity        0
invoice_date    0
price           0
customer_id     0
country         0
description     0
dtype: int64

In [61]:
# replace the descriptions with ? with Unknown

online_trans_cleaned.description.replace("?", "Unknown", inplace = True)


In [62]:
online_trans_cleaned.description.value_counts()

Unknown                               8950
CREAM HANGING HEART T-LIGHT HOLDER    2065
REGENCY CAKESTAND 3 TIER              1894
JUMBO BAG RED RETROSPOT               1659
PARTY BUNTING                         1409
                                      ... 
TEA TIME BREAKFAST BASKET                1
PINK FLOCK PHOTO FRAME                   1
DIAMANTE NECKLACE                        1
HAPPY BIRTHDAY CARD TEDDY/CAKE           1
LETTER "W" BLING KEY RING                1
Name: description, Length: 3645, dtype: int64

In [63]:
online_trans_cleaned[online_trans_cleaned.description == "?"]

# replace the ? with Unknown

,invoice,stock_code,quantity,invoice_date,price,customer_id,country,description


In [64]:
online_trans_cleaned.describe()

,quantity,price
count,408816.000000,408816.000000
mean,12.136416,3.097694
std,248.046315,19.564586
min,-80995.000000,0.000000
25%,2.000000,1.250000
50%,5.000000,1.950000
75%,12.000000,3.750000
max,80995.000000,8142.750000


## Loading Data to s3 and redshift 

In [16]:
# load the online_transactions_final data frame to s3

# connect to s3

AWS_ACCESS_KEY_ID = ""
AWS_SECRET_ACCESS_KEY = ""

s3_client = boto3.client(
    "s3",
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY
)


In [17]:
def df_to_s3(df, key, s3_bucket):
    """Function that writes a data frame as a .csv file to an s3 bucket"""

    csv_buffer = StringIO() # create buffer to temporarily store the Data Frame

    df.to_csv(csv_buffer, index=False) # code to write the data frame as csv file

    response = s3_client.put_object(
            Bucket=s3_bucket, Key=key, Body=csv_buffer.getvalue()
        ) # this code writes the temp stored csv file and writes to s3

    print("Dataframe is saved as CSV in S3 bucket.") # prints a statement to let us know the file has been written


In [18]:
# load table from s3 to redshift

AWS_S3_BUCKET = "waia-data-dump"
key = "online_transactions_transformation/sh_online_w_description.csv"

# write the file to s3 bucket

df_to_s3(online_transactions_final, key, AWS_S3_BUCKET)

Dataframe is saved as CSV in S3 bucket.


In [20]:
# close the connection

connect.close()


### Check we carried out all the tasks

In [26]:
# option 1 - read the file from s3, check the code from day3

# option 2- I loaded the changes to a table called online_transactions_cleaned

# check the new table we created and all the transformation tasks have been carried out
connect, cursor = connect_to_redshift(dbname, host, port, user, password)

query = """select *
           from bootcamp1.online_transactions_cleaned
           limit 100
           """

test = pd.read_sql(query, connect)
test.info()

connection to redshift made


,count
0,408816


In [ ]:
# make sure all transformations are in place
# anything else?!